In [1]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
from typing import Optional, Union
import pickle
import model
import configs
from torch.profiler import profile, record_function, ProfilerActivity
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
from typing import Optional, Union
import pickle
import model
import configs
import numpy as np
import pickle
import configs
from sklearn.utils import shuffle
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
from typing import Optional, Union
import configs
import pickle
import model

/Users/gl/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Define the output layer
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Decode the hidden state of the last time step
        out = self.linear(out[:, -1, :])
        return out

In [5]:
model = LSTMClassifier(52, 100, 25, 1).to(device)
with open ('small_sample', 'rb') as fp:
    data_feature = pickle.load(fp)
fp.close()

In [12]:
epochs = 100
learning_rate= 0.0002

AttributeError: module 'configs' has no attribute 'LSTM_epoch'

In [11]:

train_size = configs.training_size_scale


    
with open ('feature', 'rb') as fp:
    data_feature = pickle.load(fp)
fp.close()

with open ('target', 'rb') as fp:
    data_target = pickle.load(fp)
fp.close()

cap = int(len(data_feature)*train_size - 10000*train_size)
train_set = data_feature[:cap]
train_target = data_target[:cap]
valid_set = data_feature[-15000:-10000]
valid_target = data_target[-15000:-10000]
test_set = data_feature[-10000:]
test_target = data_target[-10000:]

train_set = torch.Tensor(train_set).to(device)
train_target = torch.Tensor(train_target).to(device)
valid_set = torch.Tensor(valid_set).to(device)
valid_target = torch.Tensor(valid_target).to(device)
test_set = torch.Tensor(test_set).to(device)
test_target = torch.Tensor(test_target).to(device)

model = LSTMClassifier(52, 100, 25, 1).to(device)
criterion = nn.CrossEntropyLoss()

batch_size = 64
def get_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in get_batches(train_set, train_target, batch_size):
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())  # Ensure labels are long type for CrossEntropyLoss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    

    model.eval()
    pred = model(valid_set)
    vloss =  criterion(pred, valid_target)

    pred_index = torch.max(pred, 1)[1]
    correct = 0
    for j in range (len(valid_target)):
        if valid_target[j][pred_index[j]] == 1:
            correct +=1
    v_acc = correct/len(valid_target)
    model.train()

    print("Finish epoch {}, loss: {}, vloss: {}, vAcc: {}".format(i, running_loss/len(train_set), vloss, v_acc))

model.eval()
pred = model(test_set)
pred_index = torch.max(pred, 1)[1]
correct = 0
for i in range (len(test_target)):
    if test_target[i][pred_index[i]] == 1:
        correct +=1

print("accruracy: ", correct/len(test_target))

#torch.save(model.state_dict(), "./models/cnn_model.pth")

: 

In [4]:
epochs = configs.cnn_epoch
learning_rate= configs.cnn_learning_rate
train_size = configs.training_size_scale

with open ('feature', 'rb') as fp:
    data_feature = pickle.load(fp)
fp.close()

with open ('target', 'rb') as fp:
    data_target = pickle.load(fp)
fp.close()

In [11]:
train_size=0.1
cap = int(len(data_feature)*train_size - 10000*train_size)
train_set = data_feature[:cap]
train_target = data_target[:cap]
valid_set = data_feature[-15000:-10000]
valid_target = data_target[-15000:-10000]
test_set = data_feature[-10000:]
test_target = data_target[-10000:]

In [6]:
train_set = torch.Tensor(train_set).to(device)
train_target = torch.Tensor(train_target).to(device)
valid_set = torch.Tensor(valid_set).to(device)
valid_target = torch.Tensor(valid_target).to(device)
test_set = torch.Tensor(test_set).to(device)
test_target = torch.Tensor(test_target).to(device)

In [7]:
model = LSTMClassifier(52, 100, 25, 1).to(device)
criterion = nn.CrossEntropyLoss()

In [8]:
batch_size = 64
def get_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in get_batches(train_set, train_target, batch_size):
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)  # Ensure labels are long type for CrossEntropyLoss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    

    model.eval()
    pred = model(valid_set)
    vloss =  criterion(pred, valid_target)

    pred_index = torch.max(pred, 1)[1]
    correct = 0
    for j in range (len(valid_target)):
        if valid_target[j][pred_index[j]] == 1:
            correct +=1
    v_acc = correct/len(valid_target)
    model.train()

    print("Finish epoch {}, loss: {}, vloss: {}, vAcc: {}".format(i, running_loss/len(train_set), vloss, v_acc))

Finish epoch 0, loss: 0.05231518510637362, vloss: 3.156096935272217, vAcc: 0.0474
Finish epoch 1, loss: 0.051222895122496774, vloss: 3.102509021759033, vAcc: 0.0876
Finish epoch 2, loss: 0.05029269986527187, vloss: 3.0528180599212646, vAcc: 0.1282
Finish epoch 3, loss: 0.04941140445427487, vloss: 3.0054376125335693, vAcc: 0.2036
Finish epoch 4, loss: 0.04849866126933663, vloss: 2.9563992023468018, vAcc: 0.2626
Finish epoch 5, loss: 0.04761372110069427, vloss: 2.9091250896453857, vAcc: 0.3046
Finish epoch 6, loss: 0.046813398285180434, vloss: 2.8628711700439453, vAcc: 0.3324
Finish epoch 7, loss: 0.04594261369559857, vloss: 2.814603328704834, vAcc: 0.35
Finish epoch 8, loss: 0.04502811213870121, vloss: 2.764235496520996, vAcc: 0.3634
Finish epoch 9, loss: 0.044141344560129005, vloss: 2.716252326965332, vAcc: 0.3628
Finish epoch 10, loss: 0.04326634530026077, vloss: 2.669110059738159, vAcc: 0.3678
Finish epoch 11, loss: 0.04243877121320503, vloss: 2.6237854957580566, vAcc: 0.3692
Finish 